### Import Required Packages and Set Options

#### Import Base Libraries

In [1]:
import os
import sys
import random

import numpy as np
import pandas as pd
import numba as nb

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from functools import partial

#### Put the Main Package Library on the PYTHONPATH

In [2]:
curdir = sys.path[0]
srcdir = os.path.join(os.path.split(curdir)[0], 'rankfm')
sys.path[0] = srcdir
srcdir

'/Users/ericlundquist/Repos/rankfm/rankfm'

#### Re-Load all Package Modules on Execution for Testing

In [3]:
%load_ext autoreload
%autoreload 2

from rankfm import RankFM
from evaluation import precision_at_k, recall_at_k

#### Set File Path Constants

In [4]:
REPO_ROOT = os.path.split(srcdir)[0]
DATA_ROOT = os.path.join(REPO_ROOT, "data/ml-100k")
print("\n".join([REPO_ROOT, DATA_ROOT]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/ml-100k


### Prepare Example Data

#### Load Users Data

In [5]:
users_df = pd.read_csv(os.path.join(DATA_ROOT, "users.csv"))
users_df['agegroup'] = pd.cut(users_df['age'], [0, 30, 45, 100], right=False, labels=False)
users_df = users_df.drop(['age', 'zip_code'], axis=1)
users_df = pd.get_dummies(users_df, prefix_sep='__', columns=['agegroup', 'gender', 'occupation'])

#### Load Items Data

In [6]:
items_df = pd.read_csv(os.path.join(DATA_ROOT, "items.csv"))
item_names = items_df[['item_id', 'item_name']]
item_names.head()

,item_id,item_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
items_df = items_df.drop(['item_name', 'release_date'], axis=1)
items_df.columns = ['item_id'] + ["genre__{}".format(col) for col in items_df.columns[1:]]

#### Load Ratings Data

In [8]:
ratings_explicit = pd.read_csv(os.path.join(DATA_ROOT, "ratings.csv"))
ratings_explicit['timestamp'] = pd.to_datetime(ratings_explicit['unix_timestamp'], origin='unix', unit='s')
ratings_explicit['positive_feedback'] = ratings_explicit.groupby('user_id')['rating'].transform(lambda c: np.where(c > c.mean(), 1, 0))
ratings_explicit = ratings_explicit.drop('unix_timestamp', axis=1)
ratings_explicit.mean()

user_id              462.48475
item_id              425.53013
rating                 3.52986
positive_feedback      0.54194
dtype: float64

#### Generate Implicit Feedback Ratings Data

In [9]:
ratings_implicit = ratings_explicit[ratings_explicit.positive_feedback == 1].reset_index(drop=True)
ratings_implicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


#### Print Final Matrix Shapes

In [10]:
print(ratings_explicit.user_id.nunique(), ratings_explicit.item_id.nunique())
print(ratings_implicit.user_id.nunique(), ratings_implicit.item_id.nunique())

print(ratings_explicit.shape)
print(ratings_implicit.shape)

943 1682
943 1483
(100000, 5)
(54194, 5)


#### Create Simple Interaction Data for Testing

In [11]:
interactions_implicit = ratings_implicit[['user_id', 'item_id']].astype(np.int32)
interactions_implicit.head()

,user_id,item_id
0,253,465
1,286,1014
2,200,222
3,224,29
4,122,387


In [12]:
user_features = users_df[users_df.user_id.isin(interactions_implicit.user_id.unique())]
item_features = items_df[items_df.item_id.isin(interactions_implicit.item_id.unique())]

In [13]:
print(interactions_implicit.shape)
print(interactions_implicit.dtypes)

(54194, 2)
user_id    int32
item_id    int32
dtype: object


In [14]:
print(user_features.shape)
print(item_features.shape)

(943, 27)
(1483, 19)


### Train the Model Using Numba

#### Initialize Internal Data Using the Main Modeling Class

In [1]:
rankfm = RankFM()

NameError: name 'RankFM' is not defined

In [ ]:
%%prun -l 10
rankfm._init_all(interactions_implicit, user_features, item_features)

#### Check the Internal Data

In [17]:
type(rankfm.interactions), rankfm.interactions.shape, rankfm.interactions.dtype, round(rankfm.interactions.nbytes / 1e6, 2)

(numpy.ndarray, (54194, 2), dtype('int32'), 0.43)

In [18]:
round(sys.getsizeof(rankfm.user_items_py) / 1e6, 2), round(sys.getsizeof(rankfm.user_items_nb) / 1e6, 2)

(0.04, 0.0)

In [19]:
print(len(rankfm.user_items_py.keys()))
rankfm.user_items_py[1]

943


array([289, 248,  49, 294,  12, 300, 254, 312, 298, 310, 276, 296, 274,
       279, 108, 292, 239, 280, 273,   0,  13, 290, 307, 303,  24, 270,
       308, 266, 252, 281, 234, 297,  99, 124, 282, 301, 269, 283, 272,
       299], dtype=int32)

In [20]:
print(len(rankfm.user_items_nb.keys()))
rankfm.user_items_nb[1]

943


array([289, 248,  49, 294,  12, 300, 254, 312, 298, 310, 276, 296, 274,
       279, 108, 292, 239, 280, 273,   0,  13, 290, 307, 303,  24, 270,
       308, 266, 252, 281, 234, 297,  99, 124, 282, 301, 269, 283, 272,
       299], dtype=int32)

#### Define the Main Internal `_fit()` Function

In [71]:
@nb.njit(cache=True)
def _fit(interactions, user_items, item_idx, regularization, learning_rate, x_uf, x_if, w_i, w_if, v_u, v_i, v_uf, v_if, epochs=1, verbose=False):
    """main internal NJIT fitting function"""
    
    P = x_uf.shape[1]
    Q = x_if.shape[1]
    F = v_i.shape[1]
    
    def isin(needle, haystack):
        for i in range(len(haystack)):
            if needle == haystack[i]:
                return True
        return False
    
    for epoch in range(epochs):
        
        shuffle_index = np.arange(len(interactions))
        np.random.shuffle(shuffle_index)
        
        interactions = interactions[shuffle_index]     
        log_likelihood = 0.0
        
        for row in range(len(interactions)):

            u = interactions[row, 0]
            i = interactions[row, 1]
            n_items = len(item_idx)

            while True:
                j = int(n_items * random.random())
                if not isin(j, user_items[u]):
                    break
            
            pu_i = w_i[i] - w_i[j]
            pu_if = np.dot(x_if[i] - x_if[j], w_if)
            pu_ui = np.dot(v_i[i] - v_i[j], v_u[u])
            pu_u_if = np.dot(x_if[i] - x_if[j], np.dot(v_if, v_u[u]))
            pu_i_uf = np.dot(x_uf[u], np.dot(v_uf, v_i[i] - v_i[j]))
            pu_uf_if = np.dot(np.dot(v_uf.T, x_uf[u]), np.dot(v_if.T, x_if[i] - x_if[j]))
            
            pairwise_utility = pu_i + pu_if + pu_ui + pu_u_if + pu_i_uf + pu_uf_if
            log_likelihood += np.log(1 / (1 + np.exp(-pairwise_utility)))
            
            d_con = 1.0 / (np.exp(pairwise_utility) + 1.0)
            d_reg = 2.0 * regularization

            d_w_i = 1.0
            d_w_j = -1.0
            d_w_if = x_if[i] - x_if[j]

            d_v_u = v_i[i] - v_i[j] + np.dot(v_if.T, x_if[i] - x_if[j])
            d_v_i = v_u[u] + np.dot(v_uf.T, x_uf[u])
            d_v_j = -v_u[u] - np.dot(v_uf.T, x_uf[u])

            d_v_uf = np.empty((P, F), np.float32)
            d_v_if = np.empty((Q, F), np.float32)

            for f in range(F):
                for p in range(P):
                    if (x_uf[u][p]) == 0.0:
                        d_v_uf[p, f] = 0.0
                    else:
                        d_v_uf[p, f] = (x_uf[u][p]) * (v_i[i][f] - v_i[j][f] + np.dot(v_if.T[f], x_if[i] - x_if[j]))
                for q in range(Q):
                    if (x_if[i][q] - x_if[j][q]) == 0.0:
                        d_v_if[q, f] = 0.0
                    else:
                        d_v_if[q, f] = (x_if[i][q] - x_if[j][q]) * (v_u[u][f] + np.dot(v_uf.T[f], x_uf[u]))

            w_i[i] += learning_rate * ((d_con * d_w_i)  - (d_reg * w_i[i]))
            w_i[j] += learning_rate * ((d_con * d_w_j)  - (d_reg * w_i[j]))
            w_if   += learning_rate * ((d_con * d_w_if) - (d_reg * w_if))
            v_u[u] += learning_rate * ((d_con * d_v_u)  - (d_reg * v_u[u]))
            v_i[i] += learning_rate * ((d_con * d_v_i)  - (d_reg * v_i[i]))
            v_i[j] += learning_rate * ((d_con * d_v_j)  - (d_reg * v_i[j]))
            v_uf   += learning_rate * ((d_con * d_v_uf) - (d_reg * v_uf))
            v_if   += learning_rate * ((d_con * d_v_if) - (d_reg * v_if))
            
        penalty = 0.0
        penalty += np.sum(regularization * np.square(w_i)) 
        penalty += np.sum(regularization * np.square(w_if))
        penalty += np.sum(regularization * np.square(v_u))
        penalty += np.sum(regularization * np.square(v_i))
        penalty += np.sum(regularization * np.square(v_uf))
        penalty += np.sum(regularization * np.square(v_if))
        
        if verbose:                
            log_likelihood = round(log_likelihood - penalty, 2)
            print("\ntraining epoch:", epoch)
            print("log likelihood:", log_likelihood)
            
    return w_i, w_if, v_u, v_i, v_uf, v_if
            
    

#### Define All Necessary Training Variables in Local Scope

In [110]:
user_to_index = rankfm.user_to_index
item_to_index = rankfm.item_to_index

item_idx = rankfm.item_idx
interactions = rankfm.interactions
user_items = rankfm.user_items_nb

x_uf = rankfm.x_uf
x_if = rankfm.x_if

w_i = rankfm.w_i
w_if = rankfm.w_if

v_u = rankfm.v_u
v_i = rankfm.v_i
v_uf = rankfm.v_uf
v_if = rankfm.v_if

regularization = rankfm.regularization
learning_rate = rankfm.learning_rate

##### all user/item index variables should be **int32**

In [73]:
item_idx.dtype, interactions.dtype, type(list(user_items.keys())[1]), user_items[1].dtype

(dtype('int32'), dtype('int32'), int, dtype('int32'))

##### all user/item features shouldbe **float32**

In [74]:
x_uf.dtype, x_if.dtype

(dtype('float32'), dtype('float32'))

In [75]:
x_uf.shape, x_if.shape

((943, 26), (1483, 18))

##### all weights/factors should be **float32**

In [76]:
w_i.dtype, w_if.dtype, v_u.dtype, v_i.dtype, v_uf.dtype, v_if.dtype

(dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'),
 dtype('float32'))

#### Attempt to Call the Main Fit Function

In [77]:
%%time
w_i, w_if, v_u, v_i, v_uf, v_if = _fit(interactions, user_items, item_idx, regularization, learning_rate, x_uf, x_if, w_i, w_if, v_u, v_i, v_uf, v_if, epochs=10, verbose=True)


training epoch: 0
log likelihood: -10925.32

training epoch: 1
log likelihood: -10886.91

training epoch: 2
log likelihood: -10648.86

training epoch: 3
log likelihood: -10740.38

training epoch: 4
log likelihood: -10826.1

training epoch: 5
log likelihood: -10723.2

training epoch: 6
log likelihood: -10796.38

training epoch: 7
log likelihood: -10539.67

training epoch: 8
log likelihood: -10644.58

training epoch: 9
log likelihood: -10543.6
CPU times: user 19.5 s, sys: 89.4 ms, total: 19.6 s
Wall time: 19.6 s


#### Define the Main Internal `_predict()` function

In [126]:
@nb.njit(cache=True)
def _predict(pairs, x_uf, x_if, w_i, w_if, v_u, v_i, v_uf, v_if):
    """internal JIT prediction function"""
    
    n_scores = len(pairs)
    scores = np.empty(n_scores, dtype=np.float32)
    
    for row in range(n_scores):

        u = pairs[row, 0]
        i = pairs[row, 1]
        
        if np.isnan(u) or np.isnan(i):
            scores[row] = np.nan
        else:
            
            u = int(u)
            i = int(i)
            
            item = w_i[i]
            item_features = np.dot(x_if[i], w_if)
            item_user = np.dot(v_i[i], v_u[u])
            user_item_features = np.dot(x_if[i], np.dot(v_if, v_u[u]))
            item_user_features = np.dot(x_uf[u], np.dot(v_uf, v_i[i]))
            feature_interactions = np.dot(np.dot(v_uf.T, x_uf[u]), np.dot(v_if.T, x_if[i]))
            
            pointwise_utility = item + item_features + item_user + user_item_features + item_user_features + feature_interactions
            scores[row] = pointwise_utility
            
    return scores

#     # generate model scores for user/item pairs found in training data and np.nan otherwise
#     pairs = zip(pred_interactions['user_idx'], pred_interactions['item_idx'])
#     scores = np.array([self._pointwise_utility(int(user), int(item)) if not (np.isnan(user) or np.isnan(item)) else np.nan for user, item in pairs])



#### Define Python Wrapper to Internal `_predict()` Function

In [127]:
def predict(pairs, cold_start='nan'):
    """calculate predicted pointwise utilities for (user, item) pairs

    :param pairs: dataframe of user/item pairs: [user_id, item_id]
    :param cold_start: whether to generate missing values ('nan') or drop ('drop') user/item pairs not found in training data
    :return: vector of real-valued model scores
    """
    
    # ensure that the model has been fit before attempting to generate predictions
    # assert self.is_fit, "you must fit the model prior to generating predictions"

    # map raw user/item identifiers to internal index positions
    pred_pairs = pd.DataFrame(pairs.copy(), columns=['user_id', 'item_id'])
    pred_pairs['user_id'] = pred_pairs['user_id'].map(user_to_index)
    pred_pairs['item_id'] = pred_pairs['item_id'].map(item_to_index)
    
    pred_pairs = pred_pairs.to_numpy().astype(np.float32)
    scores = _predict(pred_pairs, x_uf, x_if, w_i, w_if, v_u, v_i, v_uf, v_if)
    
    if cold_start == 'nan':
        return scores
    elif cold_start == 'drop':
        return scores[~np.isnan(scores)]
    else:
        raise ValueError("param [cold_start] must be set to either 'nan' or 'drop'")
 
    
    
    

In [128]:
pairs = interactions.astype(np.float32)
pairs[1, 0] = np.nan
pairs[2, 1] = np.nan
pairs[:5]

array([[252., 456.],
       [ nan, 984.],
       [199.,  nan],
       [223.,  28.],
       [121., 383.]], dtype=float32)

In [129]:
%%time

scores = predict(pairs, cold_start='nan')

CPU times: user 762 ms, sys: 513 ms, total: 1.28 s
Wall time: 2.25 s


In [116]:
pairs.shape, scores.shape

((54194, 2), (54194,))

In [117]:
scores[:5]

array([ 0.36280236,         nan,         nan,  2.81907   , -2.748325  ],
      dtype=float32)

In [122]:
test_array = np.array([1.5, 3.2, 0.9, 4.1, 2.2, 3.6])

In [125]:
np.argsort(test_array)[::-1]

array([3, 5, 1, 4, 0, 2])